In [1]:
from deepfindET.utils import copick_tools
import copick
import os
import numpy as np
from patchify import patchify
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

2025-01-07 22:52:47.744386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 22:52:47.852042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736270567.893666  119631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736270567.906363  119631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 22:52:48.003980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
copick_config_path = "working/copick.json"
copick_root = copick.from_file(copick_config_path)

runs = os.listdir('Extracted_Data/train/static/ExperimentRuns')
voxel_size = 10
tomogram_algo = 'denoised'

out_name = 'remotetargets'
out_user_id = 'deepfindET'
out_session_id = '0'

In [3]:
def get_tomogram(run_id):
    return np.array(copick_tools.get_copick_tomogram(copick_root,
                                                voxelSize = voxel_size,
                                                tomoAlgorithm=tomogram_algo,
                                                tomoID=run_id))

def get_mask(run_id):
    return np.array(copick_tools.get_copick_segmentation(copick_root.get_run(run_id),
                                                segmentationName=out_name,
                                                userID=out_user_id,
                                                sessionID=out_session_id))

def shape_t(obj):
    obj = patchify(obj, (128, 128, 128), step=128)
    obj = np.reshape(obj, (-1, obj.shape[3], obj.shape[4], obj.shape[5]))
    return obj


def tomograms():
    tomograms = np.array([shape_t(get_tomogram(i)) for i in runs])
    tomograms = np.reshape(tomograms, (-1, tomograms.shape[2], tomograms.shape[3], tomograms.shape[4]))
    tomograms = np.stack((tomograms,)*3, axis=-1)
    return tomograms
def masks():
    masks = np.array([shape_t(get_mask(i)) for i in runs])
    masks = np.reshape(masks, (-1, masks.shape[2], masks.shape[3], masks.shape[4]))
    masks = to_categorical(masks,num_classes = 7)
    return masks
    

In [4]:
y_train,y_test = train_test_split(masks(), test_size = 0.2)

In [5]:
x_train, x_test = train_test_split(tomograms(), test_size=0.2)

In [6]:
# training
np.save('working/Temp_Data/train/x_train.npy', x_train)
np.save('working/Temp_Data/train/y_train.npy', y_train)

#testing
np.save('working/Temp_Data/test/x_test.npy', x_test)
np.save('working/Temp_Data/test/y_train.npy', y_test)

# Freeing the memory
del x_train, x_test, y_train, y_test